# Swipes Table
In this notebook we're going to read in the raw swipes data exported from the accounts table and see if we can create any new variables from it

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn
print('Pandas version    : {}'.format(pd.__version__))
print('Numpy version    : {}'.format(np.__version__))
print('Matplotlib version: {}'.format(matplotlib.__version__))
print('Seaborn version   : {}'.format(seaborn.__version__))

path = '../data/'

Pandas version    : 0.20.1
Numpy version    : 1.12.1
Matplotlib version: 2.0.2
Seaborn version   : 0.7.1


In [17]:
swipes = pd.read_csv(path + 'surgeapp_swipes.raw.csv')

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', r'call to matplotlib.use() has no effect')
import pandas_profiling as pp
pp.ProfileReport(swipes)

Number of variables,6
Number of observations,5739794
Total Missing (%),15.5%
Total size in memory,262.7 MiB
Average record size in memory,48.0 B
Numeric,3
Categorical,3
Date,0
Text (Unique),0
Rejected,0
Distinct count,5735032


Recycled and Superlike have nulls, and Liked and Superlike are T/F instead of 0/1. We'll do a bit of cleanup here to prepare the data for use in the model.

In [18]:
# Liked
swipes.loc[swipes.liked != 't', 'liked'] = 0
swipes.loc[swipes.liked == 't', 'liked'] = 1

# Superlike
swipes.loc[swipes.superlike != 't', 'superlike'] = 0
swipes.loc[swipes.superlike == 't', 'superlike'] = 1

# Recycled already uses 1's for true, but null for false
swipes.loc[swipes.recycled != 1, 'recycled'] = 0
swipes['recycled'] = swipes.recycled.astype(int)

## Aggregation
Now that the data's cleaned up, let's aggregate the data to take a look at the distribution of likes and superlikes

In [19]:
# Who is the most popular?
cols = ['to_account_id', 'liked', 'superlike']
been_liked = swipes[cols].groupby('to_account_id').sum().reset_index() \
    .sort_values(['superlike', 'liked'], ascending=False)
been_liked.head(10)

,to_account_id,liked,superlike
1755,2436572,1261,28
126,153043,1367,27
521,425985,1040,25
834,708373,864,25
4026,3067257,580,21
1677,2400077,761,20
1486,2271189,1390,19
1614,2365602,1137,18
1445,2243372,1544,17
562,445218,1042,17


In [20]:
# How many have never been liked
never_been_liked = been_liked[been_liked.liked == 0]
print("{} users have never been liked".format(len(never_been_liked)))
never_been_liked.head(10)

129 users have never been liked


,to_account_id,liked,superlike
87,137449,0,0
235,217602,0,0
803,670056,0,0
907,786023,0,0
970,872455,0,0
1097,1008197,0,0
1255,2067153,0,0
1376,2194019,0,0
1530,2308885,0,0
1669,2396019,0,0


In [21]:
# But they've liked others?...
cols = ['from_account_id', 'liked', 'superlike']
has_liked = swipes[cols].groupby('from_account_id').sum().reset_index() \
    .sort_values(['superlike', 'liked'], ascending=False)
has_liked.head(10)

,from_account_id,liked,superlike
2939,2942563,552,474
1679,2445153,578,316
3163,2984619,1460,118
1840,2547723,367,75
1904,2588032,295,50
940,899111,676,44
3083,2969907,1966,42
1420,2280969,525,35
1186,2066234,546,33
2247,2758609,231,33


Fortunately no one is that sad!

In [26]:
print("{} users are very sad"
      .format(len(has_liked[has_liked.from_account_id.map(lambda x: x in never_been_liked.to_account_id)])))

0 users are very sad


Here's an example of a user with no likes:

In [27]:

swipes[(swipes.to_account_id == 137449) | (swipes.from_account_id == 137449)]

,from_account_id,to_account_id,created,liked,superlike,recycled
661654,137449,477218,2016-11-27 16:53:03.02404,0,0,0
1261001,121374,137449,2015-04-06 00:19:20,0,0,0
1261002,117163,137449,2015-04-05 14:06:48,0,0,0
1261003,128639,137449,2015-04-05 09:04:43,0,0,0
1261004,127307,137449,2015-04-05 20:05:30,0,0,0
1261005,103564,137449,2015-04-05 20:03:05,0,0,0
1261006,130265,137449,2015-04-06 05:07:02,0,0,0
1261007,138448,137449,2015-04-06 02:26:30,0,0,0
1261008,158578,137449,2015-06-21 16:54:23,0,0,0
1281933,137449,103306,2015-04-05 07:56:43,0,0,0


Let's take a look at how many swipes total there are per user (both from and to)

In [29]:
cols = ['from_account_id', 'liked']
has_swiped = swipes[cols].groupby('from_account_id').count().reset_index() \
    .rename(columns={'liked':'has_swiped'}) \
    .sort_values(['has_swiped'], ascending=False)
has_swiped.head()

,from_account_id,has_swiped
90,139736,5518
935,893247,5346
296,276012,5304
889,846121,5302
48,121374,5270


In [30]:
cols = ['to_account_id', 'liked']
been_swiped = swipes[cols].groupby('to_account_id').count().reset_index() \
    .rename(columns={'liked':'been_swiped'}) \
    .sort_values(['been_swiped'], ascending=False)
been_swiped.head()

,to_account_id,been_swiped
106,144879,3092
989,893247,3037
595,475854,3005
2598,2839815,2998
349,294889,2964


## Put these separate calculations back together in the main dataframe

In [31]:
froms = pd.merge(has_swiped, has_liked, how='outer') \
    .rename(columns={'from_account_id':'accountid', 'liked':'has_liked', 'superlike':'has_superliked'})
froms.head()

,accountid,has_swiped,has_liked,has_superliked
0,139736,5518,1169,17
1,893247,5346,1031,3
2,276012,5304,599,0
3,846121,5302,652,0
4,121374,5270,774,1


In [32]:
tos = pd.merge(been_swiped, been_liked, how='outer') \
    .rename(columns={'to_account_id':'accountid', 'liked':'been_liked', 'superlike':'been_superliked'})
tos.head()

,accountid,been_swiped,been_liked,been_superliked
0,144879,3092,496,0
1,893247,3037,414,1
2,475854,3005,1087,11
3,2839815,2998,490,0
4,294889,2964,299,0


In [42]:
acctswipes = pd.merge(froms, tos, how='outer').fillna(0).astype(int)
acctswipes.head()

,accountid,has_swiped,has_liked,has_superliked,been_swiped,been_liked,been_superliked
0,139736,5518,1169,17,2727,425,3
1,893247,5346,1031,3,3037,414,1
2,276012,5304,599,0,2869,379,1
3,846121,5302,652,0,1014,335,0
4,121374,5270,774,1,2375,768,3


## Create some new user metrics based on our aggregations

In [43]:
# Percentage of swipes of other users that were likes
acctswipes['has_liked_pct'] = acctswipes.has_liked / acctswipes.has_swiped
acctswipes[acctswipes.has_swiped > 50].sort_values('has_liked_pct', ascending=True).head()

,accountid,has_swiped,has_liked,has_superliked,been_swiped,been_liked,been_superliked,has_liked_pct
4430,2272648,68,0,0,0,0,0,0.0
4557,3054074,57,0,0,754,210,3,0.0
4033,3372032,118,0,0,0,0,0,0.0
4427,3335269,68,0,0,377,163,0,0.0
4553,3036484,57,0,0,737,133,0,0.0


In [44]:
# Who's the most popular?
acctswipes['been_liked_pct'] = acctswipes.been_liked / acctswipes.been_swiped
acctswipes[acctswipes.been_swiped > 50].sort_values('been_liked_pct', ascending=False).head()

,accountid,has_swiped,has_liked,has_superliked,been_swiped,been_liked,been_superliked,has_liked_pct,been_liked_pct
1493,3104410,1563,1563,0,109,88,2,1.000000,0.807339
3293,3025529,312,312,2,62,50,2,1.000000,0.806452
3458,3365607,259,211,0,218,166,5,0.814672,0.761468
547,3287280,2803,2801,0,78,58,0,0.999286,0.743590
5947,2670037,0,0,0,233,167,7,NaN,0.716738


In [45]:
# Who's popular with high standards?
acctswipes['likedlikes_ratio'] = acctswipes.been_liked_pct / acctswipes.has_liked_pct
acctswipes[acctswipes.has_liked > 50].sort_values('likedlikes_ratio', ascending=False).head()

,accountid,has_swiped,has_liked,has_superliked,been_swiped,been_liked,been_superliked,has_liked_pct,been_liked_pct,likedlikes_ratio
1388,2775892,1683,71,1,2,2,0,0.042187,1.000000,23.704225
882,2365602,2264,63,0,1735,1137,18,0.027827,0.655331,23.550322
206,2488781,3694,62,0,2298,836,3,0.016784,0.363795,21.675117
370,3139487,3189,73,0,653,321,4,0.022891,0.491577,21.474522
238,567622,3605,64,0,2054,781,1,0.017753,0.380234,21.417851


In [46]:
# Who's popular with low standards?
acctswipes['likedlikes_product'] = np.sqrt(np.square(acctswipes.been_liked_pct) + np.square(acctswipes.has_liked_pct))
acctswipes[(acctswipes.has_liked > 50) & (acctswipes.been_liked_pct > .4)] \
    .sort_values('likedlikes_product', ascending=False).head()

,accountid,has_swiped,has_liked,has_superliked,been_swiped,been_liked,been_superliked,has_liked_pct,been_liked_pct,likedlikes_ratio,likedlikes_product
4272,3345047,82,82,0,7,6,0,1.000000,0.857143,0.857143,1.317078
1493,3104410,1563,1563,0,109,88,2,1.000000,0.807339,0.807339,1.285223
3293,3025529,312,312,2,62,50,2,1.000000,0.806452,0.806452,1.284665
547,3287280,2803,2801,0,78,58,0,0.999286,0.743590,0.744121,1.245592
2830,3038566,550,550,0,31,22,0,1.000000,0.709677,0.709677,1.226231


## Finally, let's review the data and prepare it for use in modeling

In [47]:
# Let's take a look at all the new features we've created
pp.ProfileReport(acctswipes)

Number of variables,11
Number of observations,6070
Total Missing (%),2.6%
Total size in memory,569.1 KiB
Average record size in memory,96.0 B
Numeric,11
Categorical,0
Date,0
Text (Unique),0
Rejected,0
Distinct count,6070


In [48]:
# Let's fix the missing values first...
acctswipes.loc[pd.isnull(acctswipes.been_liked_pct), 'been_liked_pct'] = 0
acctswipes.loc[pd.isnull(acctswipes.has_liked_pct), 'has_liked_pct'] = 0
acctswipes.loc[pd.isnull(acctswipes.likedlikes_product), 'likedlikes_product'] = 0
acctswipes.loc[pd.isnull(acctswipes.likedlikes_ratio), 'likedlikes_ratio'] = 0
acctswipes.loc[acctswipes.likedlikes_ratio.map(np.isinf), 'likedlikes_ratio'] = 1

In [49]:
# Now normalize all the features (ommitting accoundid)
swipes_norm = acctswipes.copy()
del swipes_norm['accountid']
swipes_norm = (swipes_norm - swipes_norm.mean()) / (swipes_norm.max() - swipes_norm.min())
acctswipes = pd.concat([acctswipes['accountid'], swipes_norm], axis=1).head()

In [55]:
# Let's take a last look before writing it out...
pp.ProfileReport(acctswipes)

Number of variables,11
Number of observations,5
Total Missing (%),0.0%
Total size in memory,480.0 B
Average record size in memory,96.0 B
Numeric,7
Categorical,0
Date,0
Text (Unique),0
Rejected,4
Distinct count,5


After normalizing, we see that some of the parameters are so highly correlated that they'll create problems for the matrix factorization algorithm. Remove any with a correlation >.95

It's interesting that has_liked and has_liked_pct is nearly perfectly correlated (.9987), that was unexpected. has_superliked is highly correlated with has_swiped. And the two features intended to capture the relationship between popularity and pickiness are both highly correlated with been_liked_pct. However, since likedlikes_product only has a correlation of .93 under the cutoff, we'll keep it. likedlikes_ratio is right on the border (.9501) - I suppose we could keep it if we really thought it would be useful but let's see how we do without it.

In [56]:
del acctswipes['has_liked_pct']
del acctswipes['has_superliked']
del acctswipes['likedlikes_ratio']

acctswipes.to_csv(path + 'surgeapp_swipes_agg.csv', index=False)